In [5]:
#Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as pyplot

In [ ]:
#Load real esate and weather data
real_estate_data = pd.read_csv('path/to/real_estate_data.csv')
weather_data = pd.read_csv('path/to/real/real_estate_data.csv')

In [ ]:
#Merge dataset based on common feature
merged_data = pd.merge(real_estate_data, weather_data, on = 'location') #getdummy on location to encode text to numerical
merged_data = pd.get_dummies(merged_data,columns=['location'], drop_first= True)

In [ ]:
X = merged_data.loc[:,['price', 'location', 'temp', 'humidty']] #Features #find zipcode insted from top 5 States
Y = merged_data['price'] #Target variable

In [ ]:
#Split Data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2,random_state =42)

In [ ]:
#Standardize features
scaler = StandardScaler()  #
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transforma(X_test)

In [ ]:
#Principal Component Analysis (PCA)
pca = PCA(n_components =4)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


In [ ]:
#K-Means Clustering
kmeans = KMeans(n_cluster = 3, random_state =  42)
X_train_cluster = kmeans.fit_predict(X_train_scaled)
X_test_cluster = kmeans.predict(X_test_scaled)

In [ ]:
# Regression Models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state =42)
}

for model_name, model in models.items():
   # Cross- validation
   cv_results = cross_val_score(model, X_train_scaled,y_train,cv=5,scoring='neg_mean_squared_error')
   cv_mse = -cv_results.mean()
   print(f'{model_name}- Cross-Validated Mena Squared Error:{cv_mse}')

In [ ]:
#Visualization 
plt.scatter(X_test_pca[:,0], X_test_pca[;,1], c=X_text_cluster,cmap="viridis")
plt.title('PCA and Clustering Visualization')
plt.show()


In [ ]:
#Q/A 
#Things to Do:
#Explain bias and limitations
#Additional features(population/income/age)
